In [5]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13199608963335999314, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5474330824797575896
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2940617451008761336
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15695488000
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12575436461912557319
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install kaggle

In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"psriram93","key":"814fa1c415e19f988bf5db37dd4d5c42"}'}

In [11]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
!pwd

/content


In [19]:
tf.__version__

'2.3.0'

In [13]:
!kaggle datasets download -d puneet6060/intel-image-classification

 97% 337M/346M [00:08<00:00, 33.4MB/s]
100% 346M/346M [00:08<00:00, 43.8MB/s]


In [17]:
# !pip install zipfile
from zipfile import ZipFile
file_name = "intel-image-classification.zip"
with ZipFile(file_name,"r") as zipp:
  zipp.extractall()
  print("Done")

Done


In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
train_gen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [25]:
import matplotlib.pyplot as plt
%matplotlib inline
img1 = "/content/intel-image-classification/seg_train/seg_train/buildings/0.jpg"

In [26]:
plt.imread(img1).shape

(150, 150, 3)

In [29]:
training_set = train_gen.flow_from_directory("/content/intel-image-classification/seg_train/seg_train",target_size=(150,150),batch_size=64)

Found 14034 images belonging to 6 classes.


In [30]:
test_gen = ImageDataGenerator(rescale=1./255)
testing_set = test_gen.flow_from_directory("/content/intel-image-classification/seg_test/seg_test",target_size=(150,150),batch_size=64)

Found 3000 images belonging to 6 classes.


In [31]:
type(testing_set)

tensorflow.python.keras.preprocessing.image.DirectoryIterator

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [57]:
model = Sequential()
model.add(Conv2D(filters = 128,kernel_size=3,activation="relu",input_shape=[150,150,3]))
model.add(MaxPool2D(pool_size=2,strides=2))
Dropout(0.2)
model.add(Conv2D(filters = 64,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2,strides=2))
Dropout(0.2)
model.add(Conv2D(filters = 32,kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2,strides=2))
Dropout(0.2)
model.add(Flatten())
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=128,activation="relu"))
model.add(Dense(units=6,activation="softmax"))

In [58]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [59]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 148, 148, 128)     3584      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 74, 74, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 64)        73792     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 34, 34, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9248)             

In [60]:
early_stop = EarlyStopping(monitor="val_loss",patience=2)

In [61]:
model.fit(x = training_set,validation_data=testing_set,epochs=35,verbose=1)

Epoch 1/35
220/220 [==============================] - 87s 395ms/step - loss: 1.0746 - accuracy: 0.5732 - val_loss: 0.8868 - val_accuracy: 0.6770
Epoch 2/35
220/220 [==============================] - 86s 389ms/step - loss: 0.7752 - accuracy: 0.7128 - val_loss: 0.6705 - val_accuracy: 0.7523
Epoch 3/35
220/220 [==============================] - 84s 383ms/step - loss: 0.6481 - accuracy: 0.7569 - val_loss: 0.5610 - val_accuracy: 0.8010
Epoch 4/35
220/220 [==============================] - 83s 379ms/step - loss: 0.5813 - accuracy: 0.7902 - val_loss: 0.5975 - val_accuracy: 0.7873
Epoch 5/35
220/220 [==============================] - 83s 377ms/step - loss: 0.5114 - accuracy: 0.8172 - val_loss: 0.5498 - val_accuracy: 0.8113
Epoch 6/35
220/220 [==============================] - 83s 376ms/step - loss: 0.4670 - accuracy: 0.8349 - val_loss: 0.5110 - val_accuracy: 0.8270
Epoch 7/35
220/220 [==============================] - 81s 370ms/step - loss: 0.4292 - accuracy: 0.8458 - val_loss: 0.4912 - val_ac

In [62]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/intel-image-classification/seg_pred/seg_pred/25.jpg', target_size = (150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [63]:
labels = [k for k in training_set.class_indices.keys()]

In [64]:
print("predicted label is {}".format(labels[result.argmax(axis=1)[0]]))

predicted label is sea
